In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import imblearn

In [2]:
df = pd.read_csv("/content/Creditcard_data.csv")
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,575,-0.572263,0.731748,1.541254,0.150506,1.108974,0.372152,1.084879,-0.146329,-0.274447,...,-0.143508,-0.107582,-0.418263,-0.731029,0.877525,-0.364150,-0.177509,-0.256545,26.72,0
768,579,-1.296845,-0.511605,2.404726,-0.310762,-0.319551,-0.542842,-0.173310,0.260423,-1.202688,...,-0.071270,-0.161175,0.088496,0.285390,0.281069,-0.370130,0.043410,0.092318,80.00,0
769,579,1.214170,0.210481,0.484651,0.479768,-0.261955,-0.527039,0.021782,-0.106888,-0.037631,...,-0.224292,-0.594609,0.159877,0.091873,0.140964,0.227406,-0.017389,0.016030,5.98,0
770,580,1.267030,-0.071114,0.037680,0.512683,0.242392,0.705212,-0.226582,0.109483,0.657565,...,-0.164468,-0.177225,-0.222918,-1.245505,0.678360,0.525059,0.002920,-0.003333,12.36,0


In [3]:
df['Class'].unique()

array([0, 1])

In [4]:
# Separate the classes
class_0 = df[df['Class'] == 0]
class_1 = df[df['Class'] == 1]

# Determine the number of samples in each class
count_class_0, count_class_1 = class_0.shape[0], class_1.shape[0]

# Balance the classes by downsampling the majority class (if class_0 is greater than class_1)
if count_class_0 > count_class_1:
    class_0 = class_0.sample(count_class_1)
else:
    class_1 = class_1.sample(count_class_0)

# Concatenate the balanced classes
balanced_df = pd.concat([class_0, class_1])

# Shuffle the DataFrame to mix the rows
balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

# Now 'balanced_df' contains a balanced dataset with classes 0 and 1
balanced_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,239,1.167527,-0.318288,0.502464,0.033904,-0.584285,0.024592,-0.413738,0.186412,0.605945,...,-0.040544,0.092396,-0.038321,0.115915,0.319733,1.125327,-0.058855,-0.014192,10.00,0
1,548,-1.233426,-0.212441,1.839632,-1.802986,-0.493195,0.350424,-0.905316,0.844863,-1.523517,...,0.668410,1.574750,-0.176482,-0.221561,-0.058504,-0.163971,0.014670,-0.033210,24.99,0
2,574,1.257719,0.364739,0.306923,0.690638,-0.357792,-1.067481,0.094272,-0.210300,0.014455,...,-0.286856,-0.820658,0.127663,0.343128,0.221120,0.094391,-0.022189,0.030944,1.29,1
3,26,-0.529912,0.873892,1.347247,0.145457,0.414209,0.100223,0.711206,0.176066,-0.286717,...,0.046949,0.208105,-0.185548,0.001031,0.098816,-0.552904,-0.073288,0.023307,6.14,0
4,472,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
5,529,-2.000567,-2.495484,2.467149,1.140053,2.462010,0.594262,-2.110183,0.788347,0.958809,...,0.422452,1.195394,0.297836,-0.857105,-0.219322,0.861019,-0.124622,-0.171060,1.50,1
6,85,1.327884,-0.735012,1.502449,-0.544105,-1.909350,-0.712498,-1.292820,-0.002715,-0.246028,...,0.461092,1.308647,-0.046031,0.764375,0.281841,-0.063753,0.060406,0.037296,11.99,0
7,530,1.217261,-0.080646,-0.059293,-0.868862,-0.236628,-0.700159,0.143747,-0.111374,0.790809,...,0.081582,0.534693,-0.159644,0.296812,0.855793,-0.553063,0.054274,0.001410,1.00,0
8,124,-0.614586,0.975396,1.689027,-0.170483,-0.152020,-0.698487,0.726557,-0.067588,-0.126745,...,-0.261770,-0.550214,-0.006560,0.475347,-0.268825,0.022419,0.199155,-0.054699,8.93,0
9,484,-0.928088,0.398194,1.741131,0.182673,0.966387,-0.901004,0.879016,-0.156590,-0.142117,...,0.066353,0.281378,-0.257966,0.385384,0.391117,-0.453853,-0.104448,-0.125765,1.00,1


In [5]:
Z = 1.96 #95% confidence interval
p = 0.5
E = 0.1

# Calculate sample size using the formula
sample_size = int((Z**2 * p * (1 - p)) / E**2)

# Select five random samples from the balanced dataset
samples = balanced_df.sample(n=sample_size, replace = True)
samples

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,239,1.167527,-0.318288,0.502464,0.033904,-0.584285,0.024592,-0.413738,0.186412,0.605945,...,-0.040544,0.092396,-0.038321,0.115915,0.319733,1.125327,-0.058855,-0.014192,10.00,0
6,85,1.327884,-0.735012,1.502449,-0.544105,-1.909350,-0.712498,-1.292820,-0.002715,-0.246028,...,0.461092,1.308647,-0.046031,0.764375,0.281841,-0.063753,0.060406,0.037296,11.99,0
13,118,1.254914,0.350287,0.302488,0.693114,-0.371470,-1.070256,0.086781,-0.202836,0.035154,...,-0.287592,-0.832682,0.128083,0.339427,0.215944,0.094704,-0.023354,0.030892,2.69,1
12,181,-0.723637,0.154496,1.263163,0.964978,-0.027170,2.285202,1.891703,-0.141478,-0.110655,...,-0.078788,0.424114,0.206428,-0.966411,-0.761682,-0.437223,-0.051109,-0.380738,318.50,0
17,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,548,-1.233426,-0.212441,1.839632,-1.802986,-0.493195,0.350424,-0.905316,0.844863,-1.523517,...,0.668410,1.574750,-0.176482,-0.221561,-0.058504,-0.163971,0.014670,-0.033210,24.99,0
13,118,1.254914,0.350287,0.302488,0.693114,-0.371470,-1.070256,0.086781,-0.202836,0.035154,...,-0.287592,-0.832682,0.128083,0.339427,0.215944,0.094704,-0.023354,0.030892,2.69,1
8,124,-0.614586,0.975396,1.689027,-0.170483,-0.152020,-0.698487,0.726557,-0.067588,-0.126745,...,-0.261770,-0.550214,-0.006560,0.475347,-0.268825,0.022419,0.199155,-0.054699,8.93,0
3,26,-0.529912,0.873892,1.347247,0.145457,0.414209,0.100223,0.711206,0.176066,-0.286717,...,0.046949,0.208105,-0.185548,0.001031,0.098816,-0.552904,-0.073288,0.023307,6.14,0


In [6]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.combine import SMOTEENN
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [7]:
X = df.drop(columns=['Class'])
y = df['Class']
# splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, random_state=0)

In [8]:
ros = RandomOverSampler(random_state=42)
smote = SMOTE(random_state=42)
rus = RandomUnderSampler(random_state=42)
nm = NearMiss()
smote_enn = SMOTEENN(random_state=42)

In [9]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [10]:
clf_svm = SVC(random_state=42)
clf_lr = LogisticRegression(random_state=42)
clf_rf = RandomForestClassifier(random_state=42)
clf_gb = GradientBoostingClassifier(random_state=42)
clf_knn = KNeighborsClassifier()


In [11]:
samplings = [ros,smote,rus,nm,smote_enn]

In [12]:
ml_models = [clf_svm,clf_lr,clf_rf,clf_gb,clf_knn]

In [13]:
for sample_technique in samplings:
  X_resampled, y_resampled = sample_technique.fit_resample(X_train, y_train)
  for ml_model in ml_models:
      ml_model.fit(X_resampled, y_resampled)
      y_pred = ml_model.predict(X_test)
      accuracy = accuracy_score(y_test, y_pred)
      print(f"Accuracy for {sample_technique} and {ml_model}: {accuracy}")






Accuracy for RandomOverSampler(random_state=42) and SVC(random_state=42): 0.7637540453074434


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy for RandomOverSampler(random_state=42) and LogisticRegression(random_state=42): 0.919093851132686
Accuracy for RandomOverSampler(random_state=42) and RandomForestClassifier(random_state=42): 0.9935275080906149
Accuracy for RandomOverSampler(random_state=42) and GradientBoostingClassifier(random_state=42): 0.9935275080906149
Accuracy for RandomOverSampler(random_state=42) and KNeighborsClassifier(): 0.912621359223301
Accuracy for SMOTE(random_state=42) and SVC(random_state=42): 0.5889967637540453
Accuracy for SMOTE(random_state=42) and LogisticRegression(random_state=42): 0.9158576051779935


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy for SMOTE(random_state=42) and RandomForestClassifier(random_state=42): 0.9838187702265372
Accuracy for SMOTE(random_state=42) and GradientBoostingClassifier(random_state=42): 0.970873786407767
Accuracy for SMOTE(random_state=42) and KNeighborsClassifier(): 0.7378640776699029
Accuracy for RandomUnderSampler(random_state=42) and SVC(random_state=42): 0.6828478964401294
Accuracy for RandomUnderSampler(random_state=42) and LogisticRegression(random_state=42): 0.7249190938511327


...

Accuracy for RandomUnderSampler(random_state=42) and RandomForestClassifier(random_state=42): 0.43042071197411
Accuracy for RandomUnderSampler(random_state=42) and GradientBoostingClassifier(random_state=42): 0.5501618122977346
Accuracy for RandomUnderSampler(random_state=42) and KNeighborsClassifier(): 0.7443365695792881
Accuracy for NearMiss() and SVC(random_state=42): 0.3592233009708738


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy for NearMiss() and LogisticRegression(random_state=42): 0.8025889967637541
Accuracy for NearMiss() and RandomForestClassifier(random_state=42): 0.31715210355987056
Accuracy for NearMiss() and GradientBoostingClassifier(random_state=42): 0.06472491909385113
Accuracy for NearMiss() and KNeighborsClassifier(): 0.1423948220064725
Accuracy for SMOTEENN(random_state=42) and SVC(random_state=42): 0.5533980582524272
Accuracy for SMOTEENN(random_state=42) and LogisticRegression(random_state=42): 0.8220064724919094


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy for SMOTEENN(random_state=42) and RandomForestClassifier(random_state=42): 0.9546925566343042
Accuracy for SMOTEENN(random_state=42) and GradientBoostingClassifier(random_state=42): 0.8381877022653722
Accuracy for SMOTEENN(random_state=42) and KNeighborsClassifier(): 0.6537216828478964
